In [1]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import time
%matplotlib inline

project_name='cifar10-resnetV2-20'

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

#normalization
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform = transforms.Compose([transforms.ToTensor(),
                                                                             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]),
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform = transforms.Compose([transforms.ToTensor(),
                                                                             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]),
                                            download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=256, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=256, 
                                          shuffle=False)

print(train_dataset.classes)
print(len(train_dataset))

# conv = nn.Conv2d(3,1,kernel_size=3, padding=4)









  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data/
Files already downloaded and verified
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
50000


In [2]:
class ResidualBlock(nn.Module):
   def __init__(self, in_channels, channels, stride=1):
     super(ResidualBlock, self).__init__()
     conv_kwargs = {
           "kernel_size": (3, 3),
           "padding": 1,  # To ensure 3x3 conv does not reduce image size. padding=1 also works
           "bias": False
       }
     self.stride = stride
     self.in_channels = in_channels
     self.channels = channels
     self.bn1 = nn.BatchNorm2d(channels)
     self.relu = nn.ReLU()
       # This conv is in_channels -> channels and applies stride
     self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=channels, stride=stride,
                           **conv_kwargs)
     self.bn2 = nn.BatchNorm2d(channels)
       # This conv is channels -> channels
     self.conv2 = nn.Conv2d(in_channels=channels, out_channels=channels, **conv_kwargs)
  
   def strided_identity(self, x):
       # Downsample with 'nearest' method (this is striding if dims are divisible by stride)
       # Equivalently x = x[:, :, ::stride, ::stride].contiguous()
       x = nn.functional.interpolate(x, mode='nearest', scale_factor=(1/self.stride))
       # Create padding tensor for extra channels
       (b, c, h, w) = x.shape
       num_pad_channels = self.channels - self.in_channels
       pad = torch.zeros((b, num_pad_channels, h, w))
       # Append padding to the downsampled identity
       x = torch.cat((x, pad), dim=1)
       return x

   def forward(self, x):
       if self.stride > 1:
           identity = self.strided_identity(x)
       else:
        identity = x
       z = self.bn1(x)
       z = self.relu(z)
       z = self.conv1(z)
       z = self.bn2(z)
       z = self.relu(z)
       z = self.conv2(z)
       out = identity + z
       return out
      

In [3]:
class ResNetV2(nn.Module):
  def __init__(self):
    super(ResNetV2, self).__init__()

    self.input_layer = nn.Conv2d(3, 16, (3,3), padding=1)

    self.layer_1 = ResidualBlock(16,16)
    self.layer_2 = ResidualBlock(16,16)
    self.layer_3 = ResidualBlock(16,16)

    self.layer_4 = ResidualBlock(16,32, stride=2)
    self.layer_5 = ResidualBlock(32,32)
    self.layer_6 = ResidualBlock(32,32)

    self.layer_7 = ResidualBlock(32,64, stride=2)
    self.layer_8 = ResidualBlock(64,64)
    self.layer_9 = ResidualBlock(64,64)

    self.pool = nn.AdaptiveAvgPool2d((1,1))
    self.output_layer = nn.Linear(64,10)


  def forward(self, x):
    out = self.input_layer(x)
    out = self.layer_1(out)
    out = self.layer_2(out)
    out = self.layer_3(out)
    out = self.layer_4(out)
    out = self.layer_5(out)
    out = self.layer_6(out)
    out = self.layer_7(out)
    out = self.layer_8(out)
    out = self.layer_9(out)
    out = self.pool(out)
    out = out.flatten(1)
    out = out.view(out.size(0), -1)
    out = self.output_layer(out)
    return out

model = ResNetV2().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train()

# Train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()


    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)


    running_loss = 0.0
    running_corrects = 0

            # Iterate over data.
    for inputs, labels in train_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history
                with torch.set_grad_enabled():
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
    
                scheduler.step()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

    print(f'Train - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


    print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    




        






    
